# RAG System from scratch

# 1. Load Documents

In [150]:
from pypdf import PdfReader
import re
import os 

file_path ="/Users/benjamin/src/Agentic-rag/data/Accident vasculaire cérébral (AVC) – Santé publique France.pdf"

reader = PdfReader(file_path)

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)


Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)


In [151]:
text = reader.pages
len(text)

4

In [153]:
text = reader.pages[0].extract_text()
text = text.replace('\xa0', ' ')  # espaces insécables
text = re.sub(r'\n+', '\n', text)  # supprimer les sauts de ligne multiples
text = re.sub(r'Page\s+\d+\s+sur\s+\d+', '', text)
text = re.sub(r'https?://\S+', '', text)  # supprimer les URLs
text = re.sub(r'\d{2}/\d{2}/\d{4}\s+\d{2}:\d{2}', '', text)
text = text.replace('\n','')
text

'Accident vasculaire cérébral (AVC) – Santé publique FranceAccident vasculaire cérébralAccueil Maladies et traumatismesMaladies cardiovasculaires et accident vasculaire cérébralTabac info serviceAccessible à tous, le site tabac-info-service propose un accompagnementpersonnalisé et e8cace pour aider chacun à en ;nir avec la cigaretteEN SAVOIR PLUSL’accident vasculaire cérébral (AVC) est une maladie grave entrainant undé;cit neurologique lié à des lésions cérébrales d’origine vasculaires. LesAVC peuvent être ischémique ou hémorragique.Mis à jour le 17 juin 2019Nos missionsAssurer la surveillance épidémiologique des accidents vasculaires cérébraux Prévenir les risques d’accident vasculaire cérébral'

## 2.Chunks the documents (we'll split the documents into pages for the exemple purpose)

In [154]:
def extract_pages(folder : str): 
    for filename in os.listdir(folder):
        
        if filename.endswith(".pdf"):
            document_path = os.path.join(folder, filename)
            reader = PdfReader(document_path)
            
        for i,page in enumerate(reader.pages):
            text = page.extract_text()
            text = text.replace('\xa0', ' ')  # espaces insécables
            text = re.sub(r'\n+', '\n', text)  # supprimer les sauts de ligne
            text = re.sub(r'Page\s+\d+\s+sur\s+\d+', '', text)
            text = re.sub(r'https?://\S+', '', text)  # supprimer les URL
            text = re.sub(r'\d{2}/\d{2}/\d{4}\s+\d{2}:\d{2}', '', text)
            text = text.replace('\n','')
            
            yield {
                "document": document_path,
                "page": i,
                "text": text
            }

In [155]:
for doc in extract_pages("/Users/benjamin/src/Agentic-rag/data"):
    print(doc["document"], doc["page"])
    print(doc["text"])
    print("\n")
    break  

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)


Ignoring wrong pointing object 88 0 (offset 0)


/Users/benjamin/src/Agentic-rag/data/Cancers de la peau | Santé publique France.pdf 0
Cancers de la peau | Santé publique FranceCancers de la peauAccueil Maladies et traumatismesCancersCancers de la peau : les maladiesLes cancers de la peau pourraient constituer le cancer le plus fréquent enFrance. Attribuables dans plus de 85% des cas à une exposition excessiveaux ultraviolets (UV) naturels ou artiAciels, ils peuvent être évités grâce àune exposition raisonnée. Chaque année, entre 141 200 et 243 500 cassont diagnostiqués en France.Mis à jour le 13 juillet 2023Nos missionsSurveiller l’évolution épidémiologique des cancers de la peau (mélanomes cutanéset autres cancers cutanés) Participer à l’expertise sur la surveillance épidémiologique et les politiques deprévention de l’exposition aux UVDécrire et suivre les comportements et les connaissances de la population surl’exposition aux UVInformer sur les dangers d’une exposition excessive aux UV et promouvoir lesprincipaux gestes de préven

In [156]:
docs = list(extract_pages("/Users/benjamin/src/Agentic-rag/data"))
docs[0]

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong p

{'document': '/Users/benjamin/src/Agentic-rag/data/Cancers de la peau | Santé publique France.pdf',
 'page': 0,
 'text': 'Cancers de la peau | Santé publique FranceCancers de la peauAccueil Maladies et traumatismesCancersCancers de la peau : les maladiesLes cancers de la peau pourraient constituer le cancer le plus fréquent enFrance. Attribuables dans plus de 85% des cas à une exposition excessiveaux ultraviolets (UV) naturels ou artiAciels, ils peuvent être évités grâce àune exposition raisonnée. Chaque année, entre 141 200 et 243 500 cassont diagnostiqués en France.Mis à jour le 13 juillet 2023Nos missionsSurveiller l’évolution épidémiologique des cancers de la peau (mélanomes cutanéset autres cancers cutanés) Participer à l’expertise sur la surveillance épidémiologique et les politiques deprévention de l’exposition aux UVDécrire et suivre les comportements et les connaissances de la population surl’exposition aux UVInformer sur les dangers d’une exposition excessive aux UV et promo

## 3. Indexing (embedding model)

In [157]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI()

In [158]:
sample = docs[0]["text"]

In [159]:
len(sample)

917

In [58]:
response = client.embeddings.create(
    input=sample,
    model="text-embedding-3-small"
)

response

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.00874314270913601, 0.04785945639014244, 0.021163657307624817, 0.023007705807685852, 0.03655930608510971, -0.013144196942448616, -0.011846930719912052, -0.015202668495476246, 0.0038757154252380133, 0.021999912336468697, -0.0033476960379630327, -0.0914304330945015, 0.027167532593011856, -0.02037028782069683, -0.009048696607351303, 0.04262750968337059, 0.044900406152009964, 0.0020678522996604443, -0.00442785769701004, -0.012533088214695454, 0.0313916876912117, -0.006448804400861263, 0.0012751532485708594, -0.04631560295820236, 0.02079913765192032, -0.046787336468696594, 0.015770891681313515, -0.04014018923044205, 0.0033905808813869953, 0.039389707148075104, 0.017743593081831932, -0.03147745504975319, 0.002219289308413863, 0.009445381350815296, -0.02253597229719162, -0.041898466646671295, 0.012886887416243553, 0.02040245197713375, -0.02071336656808853, 0.02324357070028782, 0.012543809600174427, -0.008850354701280594, 0.00369881559163332

In [160]:
embeddings = [item.embedding for item in response.data]
len(embeddings[0]) # 1536 dimensions for text-embedding-3-small

1536

In [161]:
def encode_documents(docs):
    for doc in docs:
        response = client.embeddings.create(
            input=doc["text"],
            model="text-embedding-3-small"
        )
        yield {
            "document": doc["document"],
            "page": doc["page"],
            "embedding": response.data[0].embedding,
            "text" : doc["text"]
        }

In [ ]:
# Not working on Macos intell core

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("BAAI/bge-m3")

In [110]:
import h5py
import numpy as np

embeddings = []
documents = []
pages = []
texts = []

def write_embeddings_and_metadata(embeddings, documents, pages, text,  filename = 'embeddings.hdf5') :
    utf8_type = h5py.string_dtype(encoding='utf-8')
    with h5py.File(filename,'w') as f : 
        f.create_dataset('embeddings', data=embeddings)
        f.create_dataset('documents', data = np.array(documents, dtype=utf8_type))
        f.create_dataset('pages', data = np.array(pages, dtype='i'))
        f.create_dataset('text', data = np.array(text, dtype=utf8_type))


def normalize(embedding) :
    norm = np.linalg.norm(embedding)
    return embedding / norm

for doc in encode_documents(docs):
    embedding = normalize(doc["embedding"])
    embeddings.append(embedding)
    documents.append(doc["document"])
    pages.append(doc["page"])
    texts.append(doc["text"])
    
stacked_embeddings = np.stack(embeddings)
write_embeddings_and_metadata(stacked_embeddings, documents, pages,texts)


In [166]:
def load_index(filename='embeddings.hdf5'):
    with h5py.File(filename, 'r') as f:
        embeddings = f['embeddings'][:]
        documents = [d.decode('utf-8') if isinstance(d, bytes) else d for d in f['documents'][:]]
        pages = f['pages'][:]
        texts = [t.decode('utf-8') if isinstance(t, bytes) else t for t in f['text'][:]]
    return embeddings, documents, pages, texts


embeddings, documents, pages, texts = load_index()

In [168]:
texts

['Cancers de la peau | Santé publique FranceCancers de la peauAccueil Maladies et traumatismesCancersCancers de la peau : les maladiesLes cancers de la peau pourraient constituer le cancer le plus fréquent enFrance. Attribuables dans plus de 85% des cas à une exposition excessiveaux ultraviolets (UV) naturels ou artiAciels, ils peuvent être évités grâce àune exposition raisonnée. Chaque année, entre 141 200 et 243 500 cassont diagnostiqués en France.Mis à jour le 13 juillet 2023Nos missionsSurveiller l’évolution épidémiologique des cancers de la peau (mélanomes cutanéset autres cancers cutanés) Participer à l’expertise sur la surveillance épidémiologique et les politiques deprévention de l’exposition aux UVDécrire et suivre les comportements et les connaissances de la population surl’exposition aux UVInformer sur les dangers d’une exposition excessive aux UV et promouvoir lesprincipaux gestes de prévention',
 "Cancers de la peau | Santé publique FranceLes différents types de cancers de

## 4. Calculate cosine similarity 

In [169]:
stacked_embeddings.shape

(175, 1536)

In [170]:
query = "Quelles sont les principales causes d'AVC ?"

query_response = client.embeddings.create(
            input=query,
            model="text-embedding-3-small"
        )

query_embedding = query_response.data[0].embedding
 
len(query_embedding)

1536

In [171]:
def calculate_cosine_similarity(embeddings, query) : 
    query = normalize(query)
    return embeddings @ query.T
   

In [172]:
cosine_score = calculate_cosine_similarity(embeddings, query_embedding)
cosine_score

array([0.28208247, 0.26710168, 0.28245037, 0.3348004 , 0.28898298,
       0.26245499, 0.28107141, 0.26485179, 0.2726989 , 0.34274258,
       0.30436421, 0.30316902, 0.28432198, 0.30460565, 0.34491372,
       0.33207735, 0.30043675, 0.30326062, 0.28072865, 0.32812116,
       0.34486626, 0.27240781, 0.27645478, 0.30756742, 0.30707433,
       0.29012064, 0.26128395, 0.23938598, 0.26480305, 0.2523145 ,
       0.23053237, 0.17267559, 0.35156362, 0.31978437, 0.3048502 ,
       0.27112276, 0.27173893, 0.35440168, 0.41540357, 0.37801469,
       0.42486116, 0.50171276, 0.35756086, 0.36427697, 0.27655867,
       0.28288965, 0.33878653, 0.24481335, 0.23011761, 0.34735272,
       0.26014704, 0.2866646 , 0.29805305, 0.17049803, 0.2313324 ,
       0.26951959, 0.21744998, 0.27413172, 0.30107545, 0.22883689,
       0.33929333, 0.39815009, 0.29876478, 0.28134733, 0.30143729,
       0.26475511, 0.2111439 , 0.25204564, 0.56300022, 0.61656238,
       0.61442512, 0.49266917, 0.29038798, 0.28030958, 0.26686

In [173]:
np.argmax(cosine_score)

np.int64(69)

In [174]:
texts[69]

"Accident vasculaire cérébral (AVC) – Santé publique FranceL’accident vasculaire cérébral2: la maladieL’accident vasculaire cérébral, une pathologie handicapanteLes accidents vasculaires cérébraux (AVC) recouvrent un ensemble de syndromes semanifestant par l’apparition brutale d’un dé;cit neurologique dû à des lésionscérébrales d’origine vasculaire.  On distingue deux grands types d’AVC : Les AVC ischémiques ou infarctus cérébraux (80% des AVC) consécutifs àl’obstruction d’une artère cérébraleLes AVC hémorragiques (20% des AVC), dus à une rupture d’un vaisseau sanguincérébral. Les accidents ischémiques transitoires (AIT) sont des accidents vasculaires d’origineischémique caractérisés par la régression précoce et complète des dé;citsneurologiques, ainsi que par l’absence d’image d’infarctus cérébral.Les conséquences d'un accident vasculaire cérébral peuvent être importantes ethandicapantes. En effet, les AVC sont souvent responsables de séquelles lourdes,l'accident pouvant toucher des g

In [175]:
np.sort(cosine_score)[::-1][:3]

array([0.61656238, 0.61442512, 0.56300022])

In [183]:
def retrieve_top_document(scores, top_k, documents, pages, texts):
    ix_scores = [*enumerate(scores)]
    ix_scores = sorted(ix_scores, key=lambda x: x[1], reverse=True)
    
    return [
        (documents[ix], pages[ix], texts[ix], score)
        for ix, score in ix_scores[:top_k]
    ]
    

Next : il faut récupérer les 3 bouts de textes intéressants avec leur score (il faut que je modifie mon fichier embedding pour récupérer le bout de texte). 
Une fois que j'ai récupérer les 3 textes les plus pertinents, je les incorpore dans le prompt du générateur qui pourra fournir une réponse précise à l'utilisateur.

In [177]:
# We can retrieve the top relevant documents
retrieve_top_document(cosine_score,3)


[('/Users/benjamin/src/Agentic-rag/data/Accident vasculaire cérébral (AVC) – Santé publique France.pdf',
  np.int32(1),
  "Accident vasculaire cérébral (AVC) – Santé publique FranceL’accident vasculaire cérébral2: la maladieL’accident vasculaire cérébral, une pathologie handicapanteLes accidents vasculaires cérébraux (AVC) recouvrent un ensemble de syndromes semanifestant par l’apparition brutale d’un dé;cit neurologique dû à des lésionscérébrales d’origine vasculaire.  On distingue deux grands types d’AVC : Les AVC ischémiques ou infarctus cérébraux (80% des AVC) consécutifs àl’obstruction d’une artère cérébraleLes AVC hémorragiques (20% des AVC), dus à une rupture d’un vaisseau sanguincérébral. Les accidents ischémiques transitoires (AIT) sont des accidents vasculaires d’origineischémique caractérisés par la régression précoce et complète des dé;citsneurologiques, ainsi que par l’absence d’image d’infarctus cérébral.Les conséquences d'un accident vasculaire cérébral peuvent être i

In [178]:
def format_documents(documents, pages, texts):
    formatted_docs = []
    for doc, page, text in zip(documents, pages, texts):
        formatted_docs.append(f"Document: {doc}, Page: {page}, Text: {text}")
    return "\n".join(formatted_docs)

In [ ]:
top_docs = retrieve_top_document(cosine_score, 3)
documents, pages, texts, _ = zip(*top_docs)
context = format_documents(documents, pages, texts)


In [180]:
chain

"Document: /Users/benjamin/src/Agentic-rag/data/Accident vasculaire cérébral (AVC) – Santé publique France.pdf, Page: 1, Text: Accident vasculaire cérébral (AVC) – Santé publique FranceL’accident vasculaire cérébral2: la maladieL’accident vasculaire cérébral, une pathologie handicapanteLes accidents vasculaires cérébraux (AVC) recouvrent un ensemble de syndromes semanifestant par l’apparition brutale d’un dé;cit neurologique dû à des lésionscérébrales d’origine vasculaire.  On distingue deux grands types d’AVC : Les AVC ischémiques ou infarctus cérébraux (80% des AVC) consécutifs àl’obstruction d’une artère cérébraleLes AVC hémorragiques (20% des AVC), dus à une rupture d’un vaisseau sanguincérébral. Les accidents ischémiques transitoires (AIT) sont des accidents vasculaires d’origineischémique caractérisés par la régression précoce et complète des dé;citsneurologiques, ainsi que par l’absence d’image d’infarctus cérébral.Les conséquences d'un accident vasculaire cérébral peuvent êt

## 5. Add documents to the generator prompt

In [186]:
from openai import OpenAI
from dotenv import load_dotenv
import os   

load_dotenv()

client = OpenAI()

def generate_answer(context, question) : 
    
    prompt = """ You are a usefull assistant specialized in medicine and diseases. Your goal is to answer to the user using these documents :

    Context : 
    {context}

    Question ; 
    {question}

    Answer :

    """
    final_prompt = prompt.format(context = context, question = question)
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [{"role":"user", "content": final_prompt}],
        temperature=0.0,
    )
    return response.choices[0].message.content

def rag_pipeline(query) : 
    embeddings, documents, pages, texts = load_index()
    
    query_response = client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    )
    query_embedding = query_response.data[0].embedding
    
    cosine_score = calculate_cosine_similarity(embeddings, query_embedding)
    top_docs = retrieve_top_document(cosine_score, 3, documents, pages, texts)
    
    doc_titles, doc_pages, doc_texts, scores = zip(*top_docs)
    context = format_documents(doc_titles, doc_pages, doc_texts)
    answer = generate_answer(context, query)

    return {
        "answer": answer,
        "sources": [
            {"document": doc, "page": page, "score": float(score)}
            for doc, page, score in zip(doc_titles, doc_pages, scores)
        ]
    }




In [191]:
query = "C'est quoi le paludisme?"
answer = rag_pipeline(query)
print(answer["answer"], "\nSources:")
for source in answer["sources"]:
    print(f"Document: {source['document']}, Page: {source['page']}, Score: {source['score']}")

Le paludisme est une maladie infectieuse potentiellement mortelle causée par un parasite du genre Plasmodium, dont il existe quatre espèces principales : Plasmodium falciparum, Plasmodium vivax, Plasmodium ovale et Plasmodium malariae. Cette maladie est transmise à l'homme par la piqûre de moustiques femelles du genre Anopheles. 

Les symptômes classiques du paludisme incluent de la fièvre, un syndrome pseudo-grippal avec frissons, des céphalées, des myalgies, de la fatigue, ainsi que des manifestations gastro-intestinales comme la diarrhée et les vomissements. Dans les cas plus graves, notamment ceux causés par Plasmodium falciparum, des complications peuvent survenir, telles qu'une défaillance circulatoire, un œdème pulmonaire, une insuffisance rénale, une hypoglycémie, et des troubles neurologiques pouvant mener au coma.

Le diagnostic du paludisme repose sur un examen microscopique parasitologique, bien que des tests rapides puissent être utilisés en urgence. Le traitement doit êtr